In [1]:
%pylab inline
import pandas as pd
import nibabel as nib
import hrf_estimation as he
from scipy import stats # for z-scoring
plt.style.use('ggplot')


Populating the interactive namespace from numpy and matplotlib


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days


In [2]:
# load DataBase
df = pd.read_csv('/mnt/em9k.csv')
df = df[df.subject == 'CO'][['runID','subject', 'runType', 'TR', 'matrix_x',
                                                    'matrix_y', 'n_slices', 'nvols','frame_file_path','frame_file',
                                                    'working_vol','brain_mask']]
df

,runID,subject,runType,TR,matrix_x,matrix_y,n_slices,nvols,frame_file_path,frame_file,working_vol,brain_mask
5,142,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn001.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
6,143,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn002.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
7,144,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn003.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
8,145,CO,val,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,val001.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
9,146,CO,imgVal,2,108,108,39,232,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,imgVal.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
10,147,CO,img,2,108,108,39,232,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,img.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
11,148,CO,val,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,val002.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
12,149,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn004.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
13,150,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn005.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...
14,151,CO,trn,2,108,108,39,210,/musc.repo/mri/7T.cmrr/Presentation/Oct2011,trn006.txt,/mnt/initial_em9k_pipe/CO_nofnirt/lotusHill.7T...,/mnt/initial_em9k_pipe/CO_nofnirt//lotusHill.7...


# Apply Brain Mask

In [3]:
base_dir = '/mnt/pickles/CO/redo_7-11-15/masked_data/'
def mask_row(row):
    volume = nib.load(row['working_vol']).get_data()
    mask = nib.load(row['brain_mask']).get_data()
    volume = volume.reshape((prod(volume.shape[:3]),-1))
    mask = mask.reshape((prod(mask.shape[:3]),-1))
    masked_vol = np.array([volume[i] for i in xrange(len(volume)) if mask[i] == 1])
    print '{} voxels selected on mask'.format(masked_vol.shape)
    sf = '{}{}{}'.format(base_dir,row['subject'],str(row['runID']))
    np.save(sf,masked_vol)
    return sf + '.npy'

In [4]:
df['masked_data'] = [mask_row(row) for i, row in df.iterrows()]

(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 232) voxels selected on mask
(122136, 232) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask
(122136, 210) voxels selected on mask


In [5]:
df['masked_data']

5     /mnt/pickles/CO/redo_7-11-15/masked_data/CO142...
6     /mnt/pickles/CO/redo_7-11-15/masked_data/CO143...
7     /mnt/pickles/CO/redo_7-11-15/masked_data/CO144...
8     /mnt/pickles/CO/redo_7-11-15/masked_data/CO145...
9     /mnt/pickles/CO/redo_7-11-15/masked_data/CO146...
10    /mnt/pickles/CO/redo_7-11-15/masked_data/CO147...
11    /mnt/pickles/CO/redo_7-11-15/masked_data/CO148...
12    /mnt/pickles/CO/redo_7-11-15/masked_data/CO149...
13    /mnt/pickles/CO/redo_7-11-15/masked_data/CO150...
14    /mnt/pickles/CO/redo_7-11-15/masked_data/CO151...
15    /mnt/pickles/CO/redo_7-11-15/masked_data/CO152...
Name: masked_data, dtype: object

# Detrending`

In [6]:
filter_params = (41,3)

base_dir = '/mnt/pickles/CO/redo_7-11-15/detrended_data_savgol_{}-{}/'.format(*filter_params)

import os
if not os.path.isdir(base_dir):
    os.makedirs(base_dir)
def detrend_row(row):
    volume = numpy.load(row['masked_data'])
    voxels = he.savitzky_golay.savgol_filter(volume, filter_params[0], filter_params[1], axis=0)
    for v in xrange(voxels.shape[0]):
        voxels[v] = stats.mstats.zscore(voxels[v])
    sf = '{}{}{}'.format(base_dir,row['subject'],str(row['runID']))
    np.save(sf,voxels)
    print voxels.shape
    return sf + '.npy'



In [7]:
df['detrended_data'] = [detrend_row(row) for i, row in df.iterrows()]

(122136, 210)
(122136, 210)
(122136, 210)
(122136, 210)
(122136, 232)
(122136, 232)
(122136, 210)
(122136, 210)
(122136, 210)
(122136, 210)
(122136, 210)


In [ ]:
detrended_ts = []
for i in df.index:
    detrended_ts.append(load(df.ix[i,'detrended_data']))


In [ ]:


original_ts = []
for i in df.index:
    m = load(df.ix[i,'masked_data'])
    for v in xrange(m.shape[0]):
        m[v] = stats.mstats.zscore(m[v])
    original_ts.append(m)
    

In [ ]:
x_lines = []
cc = 0
for i in df.index:
    x_lines.append(cc+df.ix[i,'nvols'])
    cc += df.ix[i,'nvols']
print x_lines

In [ ]:
print detrended_ts[1].shape
detrended_merged = hstack(detrended_ts)

In [ ]:
orig_merged = hstack(original_ts)

In [ ]:
orig_merged.shape
from pylab import rcParams
plots = 25


In [ ]:
rcParams['figure.figsize'] = 45,10 * plots
r = random.randint(0,detrended_merged.shape[0],plots)
print r
for rrr,i in enumerate(r):
    ax = plt.subplot(plots,1,rrr+1)
    ax.set_title('voxel #{}'.format(i))
    ax.title.set_fontsize(30)
    plt.plot(orig_merged[i,:],'r',label='original (z-scored)')
    plt.plot(detrended_merged[i,:],'g',label='detrended & z-scored',alpha=1)
    for x_line in x_lines:
        plt.axvline(x_line,color='black',lw=2,alpha=.5)
    plt.legend(fontsize=20)
plt.show()

In [ ]:
rcParams['figure.figsize'] = 45,10 * plots
r = random.randint(0,detrended_merged.shape[0],plots)
print r
for rrr,i in enumerate(r):
    ax = plt.subplot(plots,1,rrr+1)
    ax.set_title('voxel #{}'.format(i))
    ax.title.set_fontsize(30)
    plt.plot(orig_merged[i,:],'r',label='original (z-scored)')
    plt.plot(detrended_merged[i,:],'g',label='detrended & z-scored',alpha=1)
    for x_line in x_lines:
        plt.axvline(x_line,color='black',lw=2,alpha=.5)
    plt.legend(fontsize=20)
plt.show()

In [ ]:
rcParams['figure.figsize'] = 45,10 * plots
r = random.randint(0,detrended_merged.shape[0],plots)
print r
for rrr,i in enumerate(r):
    ax = plt.subplot(plots,1,rrr+1)
    ax.set_title('voxel #{}'.format(i))
    ax.title.set_fontsize(30)
    plt.plot(orig_merged[i,:],'r',label='original (z-scored)')
    plt.plot(detrended_merged[i,:],'g',label='detrended & z-scored',alpha=1)
    for x_line in x_lines:
        plt.axvline(x_line,color='black',lw=2,alpha=.5)
    plt.legend(fontsize=20)
plt.show()

In [ ]:
rcParams['figure.figsize'] = 45,10 * plots
r = random.randint(0,detrended_merged.shape[0],plots)
print r
for rrr,i in enumerate(r):
    ax = plt.subplot(plots,1,rrr+1)
    ax.set_title('voxel #{}'.format(i))
    ax.title.set_fontsize(30)
    plt.plot(orig_merged[i,:],'r',label='original (z-scored)')
    plt.plot(detrended_merged[i,:],'g',label='detrended & z-scored',alpha=1)
    for x_line in x_lines:
        plt.axvline(x_line,color='black',lw=2,alpha=.5)
    plt.legend(fontsize=20)
plt.show()

In [ ]:
df.to_csv(base_dir+'masked_and_detrended_7_11_15.csv')
print 'DataFrame saved to {}'.format(base_dir+'masked_and_detrended_7_11_15')